# Load Libraries


In [7]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
# Load environment variables from the .env file
load_dotenv()

True

# Create snowflake connection with sqlalchemy

In [8]:
snowflake_account = os.getenv('snowflake_account')
snowflake_user = os.getenv('snowflake_user')
snowflake_password = os.getenv('snowflake_password')
snowflake_warehouse = os.getenv('snowflake_warehouse')
snowflake_schema = os.getenv('snowflake_schema')

engine = create_engine(
    'snowflake://{user}:{password}@{account_identifier}/'.format(
        user=snowflake_user,
        password=snowflake_password,
        account_identifier=snowflake_account,
    )
)
connection = engine.connect()


# Create Database and use it

In [9]:
target_table = 'TopicDetails'

target_database = 'CFAInstitute'

create_database_query = f"CREATE DATABASE IF NOT EXISTS {target_database}"

connection.execute(create_database_query)
use_database_query = f"USE DATABASE {target_database}"
connection.execute(use_database_query)
connection.execute("USE WAREHOUSE TEST")


# Create Table

In [10]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {target_table} (
    Title VARCHAR,
    Curriculum VARCHAR,
    Level VARCHAR,
    Topics VARCHAR,
    "Learning Outcomes Section" VARCHAR,
    Introduction VARCHAR,
    "Summary Bullets" VARIANT,
    pdf_link VARCHAR
)
"""

connection.execute(create_table_query)


# stage file

In [11]:
connection.execute(f"TRUNCATE TABLE {target_table}")

connection.execute(f"PUT file://resources/processed_content.csv @%{target_table}")


# Load data from stage

In [12]:
connection.execute(f"COPY INTO {target_table} ON_ERROR=CONTINUE FILE_FORMAT = (FIELD_OPTIONALLY_ENCLOSED_BY = '\"' SKIP_HEADER=1 PARSE_HEADER = FALSE)")